## Question and Answering

## Use Gemini LLM with a VectorDB

In [171]:
import os
import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Provide your Google API key here")

API_KEY = os.environ["GOOGLE_API_KEY"]

In [172]:
import google.generativeai as genai
genai.configure(api_key=API_KEY)
genai_models = [(idx, model) for idx,  model in enumerate(genai.list_models())]
genai_models

[(0,
  Model(name='models/chat-bison-001',
        base_model_id='',
        version='001',
        display_name='PaLM 2 Chat (Legacy)',
        description='A legacy text-only model optimized for chat conversations',
        input_token_limit=4096,
        output_token_limit=1024,
        supported_generation_methods=['generateMessage', 'countMessageTokens'],
        temperature=0.25,
        top_p=0.95,
        top_k=40)),
 (1,
  Model(name='models/text-bison-001',
        base_model_id='',
        version='001',
        display_name='PaLM 2 (Legacy)',
        description='A legacy model that understands text and generates text as an output',
        input_token_limit=8196,
        output_token_limit=1024,
        supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
        temperature=0.7,
        top_p=0.95,
        top_k=40)),
 (2,
  Model(name='models/embedding-gecko-001',
        base_model_id='',
        version='001',
        display_name='

In [173]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

gemini_embedding = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
persist_directory  = 'docs/chroma'
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function = gemini_embedding)
vectordb._collection.count()

175

In [174]:
question = "?"
docs = vectordb.similarity_search_with_relevance_scores(question, k=7)
print(docs)
# for doc in docs:    
#     print(doc.page_content)
#     print("")

[(Document(page_content='[End of Audio]  \nDuration: 69 minutes', metadata={'page': 21, 'source': 'pdf/MachineLearning-Lecture01.pdf'}), 0.7836388792148101), (Document(page_content='[End of Audio]  \nDuration: 69 minutes', metadata={'page': 21, 'source': 'pdf/MachineLearning-Lecture01.pdf'}), 0.7836388792148101), (Document(page_content="algebra I talk about today sort of seems to be going by pretty quickl y, or if you just want \nto see some of the things I'm claiming today with our proof, if you wa nt to just see some \nof those things written out in  detail, you can come to this week's discussion section.", metadata={'page': 0, 'source': 'pdf/MachineLearning-Lecture02.pdf'}), 0.7207920064528515), (Document(page_content="All right, I realize that was a fair amount of  notation, and as I proceed through the rest of \nthe lecture today, or in future weeks as well,  if some day you're looking at me write a \nsymbol and you're wondering, gee, what was th at simple lower case N again? Or w

In [175]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro", 
                             convert_system_message_to_human=True)
                            #  temperature=0.7)

## RetrievalQA chain

In [176]:
from langchain.chains import RetrievalQA

In [177]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever= vectordb.as_retriever(),
    chain_type="refine"    
)

In [178]:
question = "What are goal of this lecture?"
result = qa_chain({'query':question})

In [179]:
result['result']

"The new context does not provide any additional information about the goals of the lecture, so the original answer is still the best response to the question.\n\nThe duration of the lecture (69 minutes) is not relevant to the goals of the lecture.\n\nTherefore, the original answer is still the best response to the question:\n\n**The goal of this lecture, and indeed the entire course, is to teach students how to use machine learning tools effectively and responsibly.**\n\nThe lecturer believes that it is important to teach students how to use machine learning tools effectively because these tools are becoming increasingly powerful and are being used in a wide variety of applications. It is important that students are able to use these tools responsibly and ethically.\n\nIn this lecture, the lecturer will provide students with some general principles for using machine learning tools well. These principles include:\n\n* **Start with a clear problem definition.** What do you want your mac

## Prompt

In [180]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.  Use three sentences maximum. Keep the answer as concise as possiable. Alwasys say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [198]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 10, 'fetch_k': 50}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [199]:
question = "what is the name of the class?"
result = qa_chain.invoke({'query':question})

In [200]:
result['result']

"I'm sorry, I cannot find the answer to what the name of the class is from the provided context. Thanks for asking!"

In [204]:
result['source_documents'][5].page_content

IndexError: list index out of range